In [1]:
import cv2
from textwrap import wrap
import shutil
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
import os, warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np

C:\Users\matus\anaconda3\envs\TF\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\matus\anaconda3\envs\TF\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

In [0]:

mage_size = 224
batch_size = 16
n_classes = 3
EPOCHS = 30

train_path = 'C:\\Users\\matus\\PycharmProjects\\VisionTransformer\\data\\Training Images'

classes = {0: "Dry",
           1: "Normal",
           2: "Wet"}
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                          samplewise_center=True,
                                                          samplewise_std_normalization=True,
                                                          validation_split=0.2,
                                                          preprocessing_function=data_augment)
# set as training data

train_gen = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical',
    subset='training')

# same directory as training data

valid_gen = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical',
    subset='validation')

class_indices_mapping = train_gen.class_indices

# Initialize SMOTE
smote = SMOTE()

# Initialize empty arrays for X and y
X_train = []
y_train = []

# Fetch batches from the generator and apply SMOTE
for i in range(train_gen.__len__()):
    # Fetch a batch of data
    x_batch, y_batch = train_gen.next()

    # Append the batch to X_train and y_train
    X_train.append(x_batch)
    y_train.append(y_batch)

# Concatenate batches into a single array
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Reshape X_train if necessary
X_train_flattened = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flattened, y_train)

# Reshape data back if needed
X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0], X_train.shape[1], X_train.shape[2],
                                              X_train.shape[3])
train_sep_dir = 'C:\\Users\\matus\\PycharmProjects\\VisionTransformer\\test_folder'

from keras.utils import array_to_img

pil_img = array_to_img(X_train_resampled[80] * 224)
pil_img.save('demo.jpg')

save_dir = "test_smote_images"

# Create the directory if it doesn't exist
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)


#This function return label name 
def get_key(val):
    for key, value in class_indices_mapping.items():
        if val == value:
            return key
    return None  # Return None if the value is not found


# Loop through each image and label
for i in range(len(X_train_resampled)):
    label_index = np.argmax(y_train_resampled[i])  # Get the index of the label with the highest value
    label = get_key(label_index)  # Get the label name from the index

    # Create a subdirectory for the class if it doesn't exist
    class_dir = os.path.join(save_dir, str(label))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    # Save the image with the corresponding label as the filename
    img_filename = f"smote_{i}.jpg"
    img_path = os.path.join(class_dir, img_filename)

    # Convert the image data back to PIL image and save
    pil_img = array_to_img(X_train_resampled[i] * 255)  # Scaling back to the original range
    pil_img.save(img_path)

    print(f"Image {i + 1}/{len(X_train_resampled)} saved as {img_path}")